## Participants questionnaire data 
at location  /opt/data/optimax_sensing/Klinische\ Variabeln_Gesamt_220120_LA.sav


from columns:
- OASIS_PRE	
- BAI_PRE
- BAI_POST2
- OASIS_POST2
- BAI_POST
- OASIS_POST
- BAI_MID2
- OASIS_MID2
- BAI_MID
- OASIS_MID 




In [6]:
import pandas as pd
import re
from bs4 import UnicodeDammit

QU_PRE = "/opt/data/optimax_sensing/unipark/PRE_Klinische Variabeln_Gesamt_220120_LA.csv"
QU_MIDa = "/opt/data/optimax_sensing/unipark/MID_Gesamtfile_211213_LA.csv"
QU_MIDb = "/opt/data/optimax_sensing/unipark/MID2_Gesamt_111021_AE.csv"
QU_POSTa = "/opt/data/optimax_sensing/unipark/Post_Gesamt_20220214_LA.csv"
QU_POSTb = "/opt/data/optimax_sensing/unipark/POST2_Gesamt_291021_AE.csv"

paths = [QU_PRE, QU_MIDa, QU_MIDb, QU_POSTa, QU_POSTb]

def extract_columns(path): 
    try:
        df = pd.read_csv(path,sep = ';')
    except:
        df = pd.read_csv(path,sep = ';', encoding="windows-1252")
        df.rename(columns={'ï»¿VP': 'VP'}, inplace=True)

    cols = ["VP", "OASIS_PRE", "BAI_PRE", "BAI_POST2", "OASIS_POST2", "BAI_POST", "OASIS_POST", "BAI_MID2", "OASIS_MID2", "BAI_MID", "OASIS_MID"]
    df = df[[i for i in cols if i in set(df.columns)]]
    df["VP"] = [re.findall(r'[0-9]+', str(i), flags=0)[0] for i in df['VP']] #make participant id a numeric string 
    return df

def create_quest_df():
    df = extract_columns(paths[0])
    for path in paths[1:]:
        df = df.merge(extract_columns(path), how = 'outer', on = ['VP'])
    return df

In [7]:
df = create_quest_df()
df

,VP,OASIS_PRE,BAI_PRE,BAI_MID,OASIS_MID,BAI_MID2,OASIS_MID2,BAI_POST,OASIS_POST,BAI_POST2,OASIS_POST2
0,102,11,,,10.0,NaN,NaN,3.0,3.0,NaN,NaN
1,101,17,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,104,11,,5,2.0,NaN,NaN,6.0,2.0,NaN,NaN
3,107,7,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109,11,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
222,482,11,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,504,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,494,6,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,503,7,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import re 
print(set([int(re.findall(r'[0-9]+', i, flags=0)[0]) for i in df['VP']]))

{101, 102, 104, 105, 106, 107, 109, 113, 114, 115, 116, 117, 118, 121, 123, 124, 128, 129, 130, 131, 133, 134, 135, 136, 137, 140, 141, 142, 146, 147, 149, 150, 151, 152, 154, 155, 156, 157, 158, 160, 161, 162, 164, 166, 168, 169, 170, 171, 173, 174, 175, 177, 178, 179, 181, 182, 184, 186, 188, 189, 190, 194, 195, 196, 197, 198, 199, 200, 201, 204, 207, 208, 210, 213, 214, 215, 216, 217, 218, 219, 220, 226, 227, 229, 230, 233, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 255, 257, 258, 261, 262, 263, 264, 265, 266, 267, 268, 269, 271, 273, 275, 277, 279, 282, 283, 284, 285, 286, 288, 290, 291, 292, 294, 295, 296, 297, 298, 299, 302, 310, 311, 312, 313, 315, 316, 317, 319, 323, 325, 326, 333, 335, 336, 340, 344, 345, 346, 348, 349, 350, 351, 352, 354, 355, 366, 369, 370, 371, 372, 374, 376, 382, 383, 387, 392, 393, 394, 396, 400, 401, 404, 405, 407, 408, 409, 411, 413, 415, 416, 417, 418, 421, 423, 424, 425, 427, 428, 429, 430, 431, 433, 434, 439, 441, 443,

In [11]:

parta = ['p102', 'p104', 'p124', 'p131', 'p134', 'p141', 'p149', 'p154', 'p166', 'p170', 'p181', 'p194', 'p201', 'p208', 'p230', 'p255', 'p261', 'p262', 'p263', 'p265', 'p273', 'p284', 'p310', 'p319', 'p326', 'p333', 'p355', 'p366', 'p369', 'p376', 'p383', 'p393', 'p400']
pspart = set([str(i[1:]) for i in parta]) #ones in ps dataset 

missing = pspart - set([str(re.findall(r'[0-9]+', i, flags=0)[0]) for i in df['VP']])  #no missing participants in questionnaire dataset 

In [21]:
df = df[df['VP'].isin(pspart)]
#df = df.drop(labels='level_0', axis=1)
df.reset_index(inplace=True, drop=True)
df.to_csv('/home/jmocel/trelium/OPTIMAX-sensing/processed_data/questionnaireGT.csv')

In [22]:
df

,VP,OASIS_PRE,BAI_PRE,BAI_MID,OASIS_MID,BAI_MID2,OASIS_MID2,BAI_POST,OASIS_POST,BAI_POST2,OASIS_POST2
0,102,11,,,10.0,NaN,NaN,3.0,3.0,NaN,NaN
1,104,11,,5,2.0,NaN,NaN,6.0,2.0,NaN,NaN
2,124,4,,8,5.0,NaN,NaN,6.0,5.0,NaN,NaN
3,131,4,,5,0.0,16.0,2.0,16.0,6.0,NaN,NaN
4,134,9,,15,5.0,NaN,NaN,15.0,5.0,NaN,NaN
5,141,11,,7,7.0,NaN,NaN,7.0,2.0,NaN,NaN
6,149,5,15,11,10.0,NaN,NaN,3.0,2.0,NaN,NaN
7,154,8,28,7,6.0,NaN,NaN,9.0,5.0,NaN,NaN
8,166,10,26,10,5.0,NaN,NaN,3.0,1.0,NaN,NaN
9,181,7,14,5,5.0,NaN,NaN,2.0,1.0,NaN,NaN


## Participants demographic data 
at location  /opt/data/optimax_sensing/Klinische\ Variabeln_Gesamt_220120_LA.sav





In [3]:
# https://github.com/Roche/pyreadstat
import pandas as pd 
df = pd.read_spss("/opt/data/optimax_sensing/Klinische Variabeln_Gesamt_220120_LA.sav")
df.columns



,VP,Datum_PRE,Alter,Geschlecht,Herkunft,Wohnsituation,Wohnsituation_sonst,Familienstand,Kinder,Kinder_Anzahl,...,LEC_ClusC_PRE,LEC_ClusD_PRE,LEC_ClusE_PRE,CTQ_EmoMiss_PRE,CTQ_KörpMiss_PRE,CTQ_SexMiss_PRE,CTQ_EmoVern_PRE,CTQ_KörpVern_PRE,CTQ_Bag_PRE,SWE_PRE
0,102,2018-05-22 06:32:03,-66,NaN,-66,NaN,-66,NaN,NaN,-66,...,0.0,0.0,0.0,7.0,5.0,5.0,24.0,13.0,12.0,NaN
1,101,2018-05-25 11:17:35,-66,NaN,-66,NaN,-66,NaN,NaN,-66,...,1.0,7.0,10.0,8.0,14.0,5.0,15.0,12.0,7.0,NaN
2,104,2018-05-30 08:14:03,24,weiblich,Schweiz,andere,mit Familie,ja,nein,-99,...,2.0,3.0,2.0,18.0,5.0,5.0,19.0,11.0,9.0,NaN
3,107,2018-06-05 11:08:50,26,männlich,Deutsch,mit Partner/in,-99,ja,nein,-99,...,7.0,5.0,11.0,5.0,5.0,5.0,24.0,13.0,12.0,NaN
4,109,2018-06-06 12:40:57,24,weiblich,Schweiz,andere,Elternhaus,nein,nein,-99,...,3.0,4.0,4.0,7.0,8.0,5.0,25.0,17.0,13.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,482,2021-11-25 11:53:01,29,männlich,Schweiz,Wohngemeinschaft,-99,ja,nein,-99,...,2.0,6.0,10.0,12.0,5.0,5.0,25.0,13.0,13.0,29.0
223,504,2021-12-09 08:40:40,24,männlich,Schweiz,Wohngemeinschaft,-99,ja,nein,-99,...,0.0,0.0,0.0,5.0,5.0,8.0,25.0,13.0,13.0,34.0
224,494,2021-12-09 12:22:58,23,männlich,Schweiz,andere,Eltern,ja,nein,-99,...,2.0,5.0,6.0,5.0,5.0,5.0,24.0,13.0,15.0,27.0
225,503,2021-12-16 12:39:55,28,männlich,CH,Wohngemeinschaft,-99,nein,nein,-99,...,0.0,8.0,4.0,8.0,5.0,5.0,18.0,15.0,10.0,16.0


In [5]:
print(list(df.columns))
#alter age Geschlecht gender Herkunft nationality


['VP', 'Datum_PRE', 'Alter', 'Geschlecht', 'Herkunft', 'Wohnsituation', 'Wohnsituation_sonst', 'Familienstand', 'Kinder', 'Kinder_Anzahl', 'Bildung', 'Arbeit1', 'Teilzeit1_Prozent', 'Arbeit2', 'Teilzeit2_Prozent', 'Schicht1', 'Schicht2', 'Schicht_AnzahlWoche1', 'Schicht_AnzahlWoche2', 'Schicht_AnzahlMonat1', 'Schicht_AnzahlMonat2', 'Einkommen', 'Alkohol1', 'Alkohol2', 'Rauchen', 'Cannabis', 'Drogen1', 'Drogen2', 'Psychiater', 'Psychiater_Zeit', 'PsychTher', 'PsychTher_Zeit', 'Psychiater_idA', 'Psychiater_idA_Zeit', 'PsychTher_idA', 'PsychTher_idA_Zeit', 'Med', 'Med_Art', 'Med_Zeit', 'Sport1', 'Sport2', 'Sport3', 'Sport4', 'Sport5', 'OASIS1_PRE', 'OASIS2_PRE', 'OASIS3_PRE', 'OASIS4_PRE', 'OASIS5_PRE', 'ODSIS1_PRE', 'ODSIS2_PRE', 'ODSIS3_PRE', 'ODSIS4_PRE', 'ODSIS5_PRE', 'BAI1_PRE', 'BAI2_PRE', 'BAI3_PRE', 'BAI4_PRE', 'BAI5_PRE', 'BAI6_PRE', 'BAI7_PRE', 'BAI8_PRE', 'BAI9_PRE', 'BAI10_PRE', 'BAI11_PRE', 'BAI12_PRE', 'BAI13_PRE', 'BAI14_PRE', 'BAI15_PRE', 'BAI16_PRE', 'BAI17_PRE', 'BAI18_P

## Participants EMA data 

Cleaner dataset at: https://github.com/ExPP-Zurich/OPTIMAX-EMA/blob/main/data/optimax_ema_data.csv (needs correction of date/time)

In [ ]:
import pandas as pd
import datetime
import pickle

GESAMTFILE_b = '/opt/data/optimax_sensing/Gesamtfile_EMA_Baseline_bereinigt_29.01.22_MD.xlsx'
GESAMTFILE_m = '/opt/data/optimax_sensing/Gesamtfile_EMA_MID_29.01.22_MD.xlsx'
GESAMTFILE_p = '/opt/data/optimax_sensing/Gesamtfile_EMA_POST_29.01.22_MD.xlsx'

def create_df(path):
    df = pd.read_excel(path, engine='openpyxl')
    df = df[['VP','Date_Time_last_action (MAX) / completed (SEMA3)','Teilnehmer Mattermost']]

    #Generate placeholder values for missing columns 
    df[['fitbit_id','empatica_id']] = None
    df['platform'] = 'infer'
    df['label'] = df['VP']
    df['end_date'] = None #df['Date_Time_last_action (MAX) / completed (SEMA3)']

    #rename and order columns
    df = df[['Teilnehmer Mattermost','fitbit_id','empatica_id','VP','platform','label','Date_Time_last_action (MAX) / completed (SEMA3)','end_date']]
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']

    #drop NAs in pid column 
    df = df.dropna(subset=['pid'])
    df = df.dropna(subset=['device_id'])

    #Convert dtypes
    df = df.astype({"label": int, "pid": int})
    df['start_date'] = pd.to_datetime(df['start_date'], infer_datetime_format = True)
    #df['start_date'] = df['start_date'].dt.round('D') #round to day, yields inaccurate results 


    #group by participant and get first and last date, put in last two columns 
    end_dates = df.groupby('pid')['start_date'].max().to_frame()
    start_dates = df.groupby('pid')['start_date'].min().to_frame()


    df = df.drop_duplicates(subset=['pid']) #delete duplicate rows 

    df = df.drop(['start_date','end_date'], axis = 1)

    df = df.merge(start_dates, how = 'inner', on = ['pid'])
    df = df.merge(end_dates, how = 'inner', on = ['pid'])

    #rename columns
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']
    return df

